In [35]:
## load in relevant packages
#import pandas as pd  
import numpy as np
import gpflow

np.random.seed(1)

In [36]:
## create fake data for model
x_train = np.linspace(-3,3,20)
y_train = np.random.exponential(np.sin(X)**2)

print(x_train)
print(y_train)
x_train = x_train.reshape(-1,1)
y_train = y_train.reshape(-1,1)

[-3.         -2.68421053 -2.36842105 -2.05263158 -1.73684211 -1.42105263
 -1.10526316 -0.78947368 -0.47368421 -0.15789474  0.15789474  0.47368421
  0.78947368  1.10526316  1.42105263  1.73684211  2.05263158  2.36842105
  2.68421053  3.        ]
[1.07461729e-02 2.48468821e-01 5.57923279e-05 2.82701687e-01
 1.54373849e-01 9.47276169e-02 1.64580355e-01 2.13716147e-01
 1.05179205e-01 1.91355786e-02 1.34336610e-02 2.40526061e-01
 1.15294365e-01 1.68057852e+00 2.71515819e-02 1.07975433e+00
 4.24109001e-01 3.99003236e-01 2.94999286e-02 4.39666712e-03]


In [37]:
## build model
kernel = gpflow.kernels.RBF(1, active_dims=[0], lengthscales=1.0)
## build model
m = gpflow.models.GPR(x_train, y_train, kern=kernel)
m.likelihood.variance = 0.01

## view 
m.as_pandas_table()

,class,prior,transform,trainable,shape,fixed_shape,value
GPR/kern/lengthscales,Parameter,None,+ve,True,(),True,1.0
GPR/kern/variance,Parameter,None,+ve,True,(),True,1.0
GPR/likelihood/variance,Parameter,None,+ve,True,(),True,0.01


In [38]:
gpflow.train.ScipyOptimizer().minimize(m)
print(m.as_pandas_table())

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 11.509249
  Number of iterations: 21
  Number of functions evaluations: 24


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 11.509249
  Number of iterations: 21
  Number of functions evaluations: 24


                             class prior transform  trainable shape  \
GPR/kern/lengthscales    Parameter  None       +ve       True    ()   
GPR/kern/variance        Parameter  None       +ve       True    ()   
GPR/likelihood/variance  Parameter  None       +ve       True    ()   

                         fixed_shape               value  
GPR/kern/lengthscales           True  1.0766695513002522  
GPR/kern/variance               True  0.0837263994557618  
GPR/likelihood/variance         True   0.139101802277386  


In [68]:
## create covariance matrix for new sensor locations
x_new = np.array([4,5,6,7,8])
x_new = x_new.reshape(-1,1)
cov_mat = m.predict_f_full_cov(x_new)[1][0,:,:]

In [69]:
print(cov_mat)
print(cov_mat.shape)

[[6.79479217e-02 5.03608790e-02 1.44745580e-02 1.70529668e-03
  8.38720951e-05]
 [5.03608790e-02 8.26272104e-02 5.42679137e-02 1.49073199e-02
  1.72556559e-03]
 [1.44745580e-02 5.42679137e-02 8.37118527e-02 5.43919299e-02
  1.49132686e-02]
 [1.70529668e-03 1.49073199e-02 5.43919299e-02 8.37263645e-02
  5.43926379e-02]
 [8.38720951e-05 1.72556559e-03 1.49132686e-02 5.43926379e-02
  8.37263994e-02]]
(5, 5)


In [47]:
x_new2 = np.array([4,5])
x_new2 = x_new2.reshape(-1,1)
cov_mat2 = m.predict_f_full_cov(x_new2)[1][0,:,:]

In [50]:
print(cov_mat2)
print(np.linalg.inv(cov_mat2))
print(cov_mat2[0,0])

[[0.06794792 0.05036088]
 [0.05036088 0.08262721]]
[[ 26.84332206 -16.36087299]
 [-16.36087299  22.07442242]]
0.06794792168481675


In [54]:
## example: thought exercise
## 5 sensor locations: S = [4,5,6,7,8]
## total number of sensors to find: k = 2
## for each iteration (1:k)
## 1st iteration:
## store_val = []
## for each possible sensor location (y) not in A (S):
    ## calculate: corr(y)^2 - corr(yA)*np.linalg.inv(corr(AA))*corr(Ay) / corr(y)^2 - corr(ynA)*np.linalg.inv(corr(nAA))*corr(nAy)
## select max(store_val)
## append y to A

## possible sensor locations
S = x_new
## number of sensors to find
k = 2
## selected sensors
A = np.array([])


In [ ]:
## x is np.array or single value
def generate_cov_mat(x):
    x = np.array(x).reshape(-1,1)
    cov_mat = m.predict_f_full_cov(x)[1][0,:,:]
    return(cov_mat)

In [56]:
for j in range(k):
    Y = S[np.isin(S, A, invert=True)]
    print(S)
    for y in Y:
        generate_cov_mat(y)^2 - 

0
1


In [62]:
A = np.array([5])
y = S[np.isin(S, A, invert=True)]
print(y)

[4 6 7 8]


In [66]:
x = np.array([4])
x = x.reshape(-1,1)
cov_mat = m.predict_f_full_cov(x)[1][0,:,:]

In [67]:
print(cov_mat)

[[0.06794792]]


In [72]:
## x is np.array or single value
def generate_cov_mat(x):
    x = np.array(x).reshape(-1,1)
    cov_mat = m.predict_f_full_cov(x)[1][0,:,:]
    return(cov_mat)

In [75]:
x = np.array([4,6])
#x = 6
print(x)
generate_cov_mat(x)

[4 6]


array([[0.06794792, 0.01447456],
       [0.01447456, 0.08371185]])